In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a4cf216d0e16829d6e57b7ec6a011aa2327f39cab7940dfbcb398559e0f1c761
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.master("local").appName("linear_regression_model").getOrCreate()

In [ ]:
real_estate = spark.read.option("inferschema","true").csv("/content/Real estate.csv",header=True)

In [ ]:
real_estate.printSchema()

root
 |-- No: integer (nullable = true)
 |-- X1 transaction date: double (nullable = true)
 |-- X2 house age: double (nullable = true)
 |-- X3 distance to the nearest MRT station: double (nullable = true)
 |-- X4 number of convenience stores: integer (nullable = true)
 |-- X5 latitude: double (nullable = true)
 |-- X6 longitude: double (nullable = true)
 |-- Y house price of unit area: double (nullable = true)



In [ ]:
real_estate.show(2)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
| No|X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
|  1|           2012.917|        32.0|                              84.87882|                             10|   24.98298|   121.54024|                      37.9|
|  2|           2012.917|        19.5|                              306.5947|                              9|   24.98034|   121.53951|                      42.2|
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
only showing top 2 rows



In [ ]:
real_estate.describe().show()

+-------+-----------------+-------------------+------------------+--------------------------------------+-------------------------------+--------------------+--------------------+--------------------------+
|summary|               No|X1 transaction date|      X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|         X5 latitude|        X6 longitude|Y house price of unit area|
+-------+-----------------+-------------------+------------------+--------------------------------------+-------------------------------+--------------------+--------------------+--------------------------+
|  count|              414|                414|               414|                                   414|                            414|                 414|                 414|                       414|
|   mean|            207.5| 2013.1489710144933| 17.71256038647343|                    1083.8856889130436|              4.094202898550725|  24.969030072463745|  121.53336108

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[
    'X1 transaction date',
 'X2 house age',
 'X3 distance to the nearest MRT station',
 'X4 number of convenience stores',
 'X5 latitude',
 'X6 longitude'],
 outputCol='features')


In [ ]:
data_set = assembler.transform(real_estate)
data_set.select(['features','Y house price of unit area']).show(2)

+--------------------+--------------------------+
|            features|Y house price of unit area|
+--------------------+--------------------------+
|[2012.917,32.0,84...|                      37.9|
|[2012.917,19.5,30...|                      42.2|
+--------------------+--------------------------+
only showing top 2 rows



In [ ]:
train_data,test_data =data_set.randomSplit([0.7,0.3])

In [ ]:
train_data.show(truncate=False)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+------------------------------------------------+
|No |X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|features                                        |
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+------------------------------------------------+
|1  |2012.917           |32.0        |84.87882                              |10                             |24.98298   |121.54024   |37.9                      |[2012.917,32.0,84.87882,10.0,24.98298,121.54024]|
|2  |2012.917           |19.5        |306.5947                              |9                              |24.98034   |121.53951   |42.2                  

In [ ]:
test_data.show(truncate=False)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+-----------------------------------------------+
|No |X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|features                                       |
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+-----------------------------------------------+
|9  |2013.5             |31.7        |5512.038                              |1                              |24.95095   |121.48458   |18.8                      |[2013.5,31.7,5512.038,1.0,24.95095,121.48458]  |
|11 |2013.083           |34.8        |405.2134                              |1                              |24.97349   |121.53372   |41.4                      

In [ ]:
from pyspark.ml.regression import LinearRegression
lr= LinearRegression(labelCol= 'Y house price of unit area')
lrModel=lr.fit(train_data)

In [ ]:
test_stats = lrModel.evaluate(test_data)
print(f"RMSE: {test_stats.rootMeanSquaredError}")
print(f"R2: {test_stats.r2}")
print(f"R2: {test_stats.meanSquaredError}")

RMSE: 7.328610335531405
R2: 0.6608529414781106
R2: 53.708529450057725
